In [ ]:
!pip install uv

In [ ]:
!uv pip install onnxruntime-gpu -i https://aiinfra.pkgs.visualstudio.com/PublicPackages/_packaging/onnxruntime-cuda-12/pypi/simple/ --system --quiet
!uv pip install fastembed-gpu huggingface_hub --system --quiet

In [ ]:
from fastembed import TextEmbedding, SparseTextEmbedding,SparseEmbedding
from kaggle_secrets import UserSecretsClient
from datasets import Dataset
from typing import List
import huggingface_hub
import polars as pl
import numpy as np
import tqdm
import ast
import os

In [ ]:
user_secrets = UserSecretsClient()
hf_toke_write = user_secrets.get_secret("hf_toke_write")
os.environ["HF_TOKEN"] = hf_toke_write

In [ ]:
huggingface_hub.login(hf_toke_write)

In [ ]:
def process_data_structure(row):
    doc_template = """
Title: {title}
Ingredients:\n{ingredients}
Directions:\n{directions}
"""
    new_row = list()

    for value in row:
        if isinstance(value, str):
            if value.startswith('[') and value.endswith(']'):
                string_list_value = ast.literal_eval(value)
                list_value= [item.strip() for item in string_list_value]
                new_row.append(list_value)
            else:
                new_row.append(value.strip())
        else:
            new_row.append(value)

    new_row[2] = "\n".join(new_row[2])
    new_row[3] = "\n".join(new_row[3])

    new_row.append({"title": new_row[1],"NER": new_row[6]})
    new_row.append(doc_template.format(title=new_row[1],ingredients=new_row[2],directions=new_row[3]).strip())

    return tuple(new_row)

In [ ]:
model = TextEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2", providers=["CUDAExecutionProvider"])
sparse_model = SparseTextEmbedding(model_name="Qdrant/bm42-all-minilm-l6-v2-attentions", providers=["CUDAExecutionProvider"])

In [ ]:
df = (
    pl.read_parquet('hf://datasets/rk404/recipe_short/final_recipes.parquet')
    .map_rows(process_data_structure)
    .rename({
        "column_0": "id",
        "column_1": "title",
        "column_2": "ingredients",
        "column_3": "directions",
        "column_4": "link",
        "column_5": "source",
        "column_6": "NER",
        "column_7": "metadata",
        "column_8": "document"
    })
)

data = df.to_dict(as_series=False)

In [ ]:
embeddings_list: List[np.array] = list(model.embed(tqdm.tqdm(data['document']), batch_size=100))

In [ ]:
sparse_embeddings_list: List[SparseEmbedding] = list(sparse_model.embed(tqdm.tqdm(data['document']), batch_size=100))

In [ ]:
data['all-MiniLM-L6-v2'] = embeddings_list

In [ ]:
data['bm42-all-minilm-l6-v2-attentions'] = [i.as_object() for i in sparse_embeddings_list]

In [ ]:
ds = Dataset.from_dict(data)

In [ ]:
ds.push_to_hub("otacilio-psf/recipe_short_dense_and_sparse_embeddings")